<a href="https://colab.research.google.com/github/leehgkor/lhg/blob/main/%EA%B5%AC%EA%B2%B0_%EC%84%A0%EC%96%B4%EB%A7%90%EC%96%B4%EB%AF%B8_%ED%8C%A8%ED%84%B4_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 개별 ep 사이의 연결 패턴 정리(앞서 만든 "화엄_structured.json" 파일을 업로드 해 사용)

# 필요한 라이브러리
import json
from collections import defaultdict
import pandas as pd
from google.colab import files

# 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# JSON 파일 로드
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

sentences = data["sentence"]

# Ep-Ep 연결 패턴 수집
ep_connections = defaultdict(int)

for sentence in sentences:
    words = sentence.get("word", [])
    if not words:
        continue

    current_form = None
    current_word = []

    for token in words + [{"form": None}]:  # 마지막 어절 flush
        if token["form"] != current_form:
            if current_word:
                # 어절 내 Ep 형태만 추출
                ep_list = []
                for w in current_word:
                    pos_tags = w["pos"].split("+")
                    if "Ep" in pos_tags:
                        ep_list.append(w["gugeol"])

                # Ep → Ep 직접 연결된 경우만 저장
                for i in range(len(ep_list) - 1):
                    pair = (ep_list[i], ep_list[i + 1])
                    ep_connections[pair] += 1

            current_form = token["form"]
            current_word = []

        if token["form"] is not None:
            current_word.append(token)

# 정렬된 결과 생성
sorted_ep = sorted(ep_connections.items(), key=lambda x: x[1], reverse=True)

# 텍스트 줄로 변환
lines = [
    f"[Ep: {a}] → [Ep: {b}]  ({count}회)"
    for (a, b), count in sorted_ep
]

# 텍스트 파일 저장
txt_filename = "화엄_Ep구결자_직접연결_시퀀스.txt"
with open(txt_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

# 다운로드 제공
files.download(txt_filename)


Saving 화엄_structured.json to 화엄_structured.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# ep 시퀀스 정리(앞서 만든 "화엄_structured.json" 파일을 업로드 해 사용)

# 필요한 라이브러리
import json
from collections import Counter
from google.colab import files

# 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# JSON 파일 로딩
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

sentences = data["sentence"]

# Ep 연속 시퀀스 추출
ep_strict_counter = Counter()

for sentence in sentences:
    words = sentence.get("word", [])
    if not words:
        continue

    current_form = None
    current_word = []

    for token in words + [{"form": None}]:  # 어절 flush
        if token["form"] != current_form:
            if current_word:
                current_seq = []
                for w in current_word:
                    pos_tags = w["pos"].split("+")
                    if "Ep" in pos_tags:
                        current_seq.append(w["gugeol"])
                    else:
                        if current_seq:
                            pattern = "→".join(current_seq)
                            ep_strict_counter[pattern] += 1
                            current_seq = []
                if current_seq:
                    pattern = "→".join(current_seq)
                    ep_strict_counter[pattern] += 1

            current_form = token["form"]
            current_word = []

        if token["form"] is not None:
            current_word.append(token)

# 정렬 및 출력 형식 변경 (stem → ... → ending)
sorted_patterns = sorted(ep_strict_counter.items(), key=lambda x: x[1], reverse=True)
lines = [f"stem → {pattern} → ending  ({count}회)" for pattern, count in sorted_patterns]

# 텍스트 파일 저장
txt_filename = "화엄_Ep구결자_직접연결_stem_ending_포함.txt"
with open(txt_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

# 다운로드
files.download(txt_filename)


Saving 화엄_structured.json to 화엄_structured (1).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ep 시퀀스 정리_화엄에만 나오는 패턴에 [*] 표시(앞서 만든 "화엄/화소/유가/금광/구인_Ep구결자_직접연결_stem_ending_포함.txt" 파일을 업로드 해 사용)

import re
import unicodedata
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 파일 이름 지정
files_by_name = {
    "화엄": "화엄_Ep구결자_직접연결_stem_ending_포함.txt",
    "화소": "화소_Ep구결자_직접연결_stem_ending_포함.txt",
    "유가": "유가_Ep구결자_직접연결_stem_ending_포함.txt",
    "금광": "금광_Ep구결자_직접연결_stem_ending_포함.txt",
    "구인": "구인_Ep구결자_직접연결_stem_ending_포함.txt",
}

# 유틸 함수: 빈도 정보 제거 + 정규화 + 공백 제거
def remove_frequency_info(line: str) -> str:
    return re.sub(r"\s*\(\d+회\)", "", line).strip()

def normalize_line(line: str) -> str:
    return unicodedata.normalize("NFKC", line).replace(" ", "").strip()

# 각 파일에서 비교용 시퀀스 추출
def extract_normalized_patterns(filepath):
    patterns = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            norm = normalize_line(remove_frequency_info(line))
            if norm.startswith("stem→") and "→ending" in norm:
                patterns.add(norm)
    return patterns

# 모든 문헌에서 정규화된 시퀀스 패턴 추출
norm_patterns_by_text = {
    name: extract_normalized_patterns(fname)
    for name, fname in files_by_name.items()
}

# 비교 대상: 화엄 이외의 문헌에서 나온 패턴 통합
norm_patterns_other = (
    norm_patterns_by_text["화소"]
    | norm_patterns_by_text["유가"]
    | norm_patterns_by_text["금광"]
    | norm_patterns_by_text["구인"]
)

# 화엄 원본 라인 다시 불러오기
with open(files_by_name["화엄"], "r", encoding="utf-8") as f:
    hwaeom_lines = f.readlines()

# [*] 표식 부여
final_lines = []
for line in hwaeom_lines:
    norm_line = normalize_line(remove_frequency_info(line))
    if norm_line.startswith("stem→") and "→ending" in norm_line:
        if norm_line not in norm_patterns_other:
            final_lines.append(line.rstrip() + "  [*]\n")
        else:
            final_lines.append(line)
    else:
        final_lines.append(line)

# 결과 저장 및 다운로드
output_path = "화엄_Ep구결자_직접연결_stem_ending_포함_고유표시.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.writelines(final_lines)

files.download(output_path)


Saving 구인_Ep구결자_직접연결_stem_ending_포함.txt to 구인_Ep구결자_직접연결_stem_ending_포함 (1).txt
Saving 금광_Ep구결자_직접연결_stem_ending_포함.txt to 금광_Ep구결자_직접연결_stem_ending_포함 (2).txt
Saving 유가_Ep구결자_직접연결_stem_ending_포함.txt to 유가_Ep구결자_직접연결_stem_ending_포함 (2).txt
Saving 화소_Ep구결자_직접연결_stem_ending_포함.txt to 화소_Ep구결자_직접연결_stem_ending_포함 (2).txt
Saving 화엄_Ep구결자_직접연결_stem_ending_포함.txt to 화엄_Ep구결자_직접연결_stem_ending_포함 (3).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ep 시퀀스 정리_화엄에 나오지 않는 패턴 정리(앞서 만든 "화엄/화소/유가/금광/구인_Ep구결자_직접연결_stem_ending_포함.txt" 파일을 업로드 해 사용)

# 필요한 라이브러리
import re
import unicodedata
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 파일 이름 매핑
files_by_name = {
    "화엄": "화엄_Ep구결자_직접연결_stem_ending_포함.txt",
    "화소": "화소_Ep구결자_직접연결_stem_ending_포함.txt",
    "유가": "유가_Ep구결자_직접연결_stem_ending_포함.txt",
    "금광": "금광_Ep구결자_직접연결_stem_ending_포함.txt",
    "구인": "구인_Ep구결자_직접연결_stem_ending_포함.txt",
}

# 함수 정의
def remove_frequency_info(line: str) -> str:
    return re.sub(r"\s*\(\d+회\)", "", line).strip()

def normalize_line(line: str) -> str:
    return unicodedata.normalize("NFKC", line).replace(" ", "").strip()

def extract_raw_lines(filepath):
    lines = set()
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("stem") and "→ending" in line:
                lines.add(line.strip())
    return lines

# 화엄 기준 시퀀스 정리
hwaeom_norm_set = set(
    normalize_line(remove_frequency_info(line))
    for line in extract_raw_lines(files_by_name["화엄"])
)

# 다른 문헌에서 화엄에 없는 시퀀스를 수집하고 출처 병기
unseen_with_source = {}

for name in ["유가", "화소", "금광", "구인"]:
    with open(files_by_name[name], "r", encoding="utf-8") as f:
        for line in f:
            cleaned_line = remove_frequency_info(line)
            norm_line = normalize_line(cleaned_line)
            if norm_line.startswith("stem→") and "→ending" in norm_line:
                if norm_line not in hwaeom_norm_set:
                    if cleaned_line not in unseen_with_source:
                        unseen_with_source[cleaned_line] = set()
                    unseen_with_source[cleaned_line].add(name)

# 정렬 및 저장
lines_with_sources = [
    f"{line}  [{'/'.join(sorted(sources))}]"
    for line, sources in unseen_with_source.items()
]
lines_with_sources_sorted = sorted(lines_with_sources)

output_path = "화엄에_없는_Ep_시퀀스_출처포함.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for line in lines_with_sources_sorted:
        f.write(line + "\n")

files.download(output_path)


Saving 구인_Ep구결자_직접연결_stem_ending_포함.txt to 구인_Ep구결자_직접연결_stem_ending_포함 (2).txt
Saving 금광_Ep구결자_직접연결_stem_ending_포함.txt to 금광_Ep구결자_직접연결_stem_ending_포함 (3).txt
Saving 유가_Ep구결자_직접연결_stem_ending_포함.txt to 유가_Ep구결자_직접연결_stem_ending_포함 (3).txt
Saving 화소_Ep구결자_직접연결_stem_ending_포함.txt to 화소_Ep구결자_직접연결_stem_ending_포함 (3).txt
Saving 화엄_Ep구결자_직접연결_stem_ending_포함.txt to 화엄_Ep구결자_직접연결_stem_ending_포함 (4).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 전체 문헌들에 나오는 선어말어미 결합 연쇄 정리(앞서 만든 "구인/화엄/화소/유가/금광_structured.json" 업로드 해 사용).

# 필요한 라이브러리
import json
import re
from collections import Counter
from google.colab import files

# 파일 업로드
uploaded = files.upload()
uploaded_files = list(uploaded.keys())

# 전체 파일 대상으로 Ep 시퀀스 추출
total_ep_counter = Counter()

for fname in uploaded_files:
    with open(fname, "r", encoding="utf-8") as f:
        data = json.load(f)

    sentences = data.get("sentence", [])

    for sentence in sentences:
        words = sentence.get("word", [])
        if not words:
            continue

        current_form = None
        current_word = []

        for token in words + [{"form": None}]:  # 어절 flush
            if token["form"] != current_form:
                if current_word:
                    current_seq = []
                    for w in current_word:
                        pos_tags = w["pos"].split("+")
                        if "Ep" in pos_tags:
                            current_seq.append(w["gugeol"])
                        else:
                            if current_seq:
                                pattern = "→".join(current_seq)
                                total_ep_counter[pattern] += 1
                                current_seq = []
                    if current_seq:
                        pattern = "→".join(current_seq)
                        total_ep_counter[pattern] += 1

                current_form = token["form"]
                current_word = []

            if token["form"] is not None:
                current_word.append(token)

# 결과 정렬 및 출력
sorted_patterns = sorted(total_ep_counter.items(), key=lambda x: x[1], reverse=True)
lines = [f"stem → {pattern} → ending  ({count}회)" for pattern, count in sorted_patterns]

# 결과 저장
output_filename = "전체문헌_Ep구결자_직접연결_stem_ending_포함.txt"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

# 다운로드
files.download(output_filename)


Saving 구인_structured.json to 구인_structured (1).json
Saving 금광_structured.json to 금광_structured (1).json
Saving 유가_structured.json to 유가_structured (1).json
Saving 화소_structured.json to 화소_structured (1).json
Saving 화엄_structured.json to 화엄_structured (1).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 전체 문헌에 중복 출현한 시퀀스 정리(앞서 만든 "구인/화엄/화소/유가/금광_structured.json" 업로드 해 사용).

# 필요한 라이브러리
import json
from collections import defaultdict, Counter
from google.colab import files

# 파일 업로드
uploaded = files.upload()
uploaded_files = list(uploaded.keys())

# 문헌별 Ep 시퀀스 수집
ep_sets_by_file = {}
ep_counts_by_file = {}

for fname in uploaded_files:
    with open(fname, "r", encoding="utf-8") as f:
        data = json.load(f)

    sentences = data.get("sentence", [])
    ep_counter = Counter()

    for sentence in sentences:
        words = sentence.get("word", [])
        if not words:
            continue

        current_form = None
        current_word = []

        for token in words + [{"form": None}]:  # 어절 flush
            if token["form"] != current_form:
                if current_word:
                    current_seq = []
                    for w in current_word:
                        pos_tags = w["pos"].split("+")
                        if "Ep" in pos_tags:
                            current_seq.append(w["gugeol"])
                        else:
                            if current_seq:
                                pattern = "→".join(current_seq)
                                ep_counter[pattern] += 1
                                current_seq = []
                    if current_seq:
                        pattern = "→".join(current_seq)
                        ep_counter[pattern] += 1

                current_form = token["form"]
                current_word = []

            if token["form"] is not None:
                current_word.append(token)

    ep_sets_by_file[fname] = set(ep_counter)
    ep_counts_by_file[fname] = ep_counter

# 교집합 추출
common_patterns = set.intersection(*ep_sets_by_file.values())

# 결과 정리
lines = []
for pattern in sorted(common_patterns):
    line = f"stem → {pattern} → ending"
    # 문헌별 빈도 병기 (선택)
    counts = [f"{name}: {ep_counts_by_file[name][pattern]}회" for name in uploaded_files]
    line += "  [" + "; ".join(counts) + "]"
    lines.append(line)

# 텍스트 저장
output_path = "공통_Ep_시퀀스_전체문헌.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

files.download(output_path)


Saving 구인_structured.json to 구인_structured (2).json
Saving 금광_structured.json to 금광_structured (2).json
Saving 유가_structured.json to 유가_structured (2).json
Saving 화소_structured.json to 화소_structured (2).json
Saving 화엄_structured.json to 화엄_structured (2).json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>